In [1]:
%pwd

'C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds'

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, regularizers


train_dir = 'C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\training-images'
val_dir = 'C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\testing-images'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 13956 images belonging to 2 classes.
Found 1089 images belonging to 2 classes.


In [5]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Add custom classification layers
x = Flatten()(base_model.output)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)  # L2 regularization
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

In [8]:
model = Model(inputs=base_model.input, outputs=output)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

C:\Users\redfr\elevenvenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7568 - loss: 2.7586

C:\Users\redfr\elevenvenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


437/437 ━━━━━━━━━━━━━━━━━━━━ 530s 1s/step - accuracy: 0.7569 - loss: 2.7561 - val_accuracy: 0.5519 - val_loss: 1.3681
Epoch 2/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 526s 1s/step - accuracy: 0.8773 - loss: 0.7059 - val_accuracy: 0.5234 - val_loss: 1.4100
Epoch 3/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 523s 1s/step - accuracy: 0.8840 - loss: 0.5196 - val_accuracy: 0.5629 - val_loss: 1.1151
Epoch 4/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 522s 1s/step - accuracy: 0.8885 - loss: 0.4449 - val_accuracy: 0.5583 - val_loss: 1.0433
Epoch 5/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 522s 1s/step - accuracy: 0.8908 - loss: 0.4035 - val_accuracy: 0.5941 - val_loss: 0.9353
Epoch 6/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 524s 1s/step - accuracy: 0.8916 - loss: 0.3873 - val_accuracy: 0.5785 - val_loss: 0.9767
Epoch 7/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 525s 1s/step - accuracy: 0.8940 - loss: 0.3721 - val_accuracy: 0.5574 - val_loss: 1.1010
Epoch 8/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 524s 1s/step - accuracy: 0.8982 - loss: 0.3571 - val_accuracy: 0.556

In [11]:
model.save('deepfake_classifier.h5')